In [2]:
import gym
import copy
import numpy as np
import torch
import sys 
sys.path.append("..")
sys.path.append("../MCTS")
from monteCarlo import MCTree
import tools
from tools import get_args, registration_envs
registration_envs()

../pretrained_models/PCT_setting1.pt
Loading pre-train upper model ../pretrained_models/PCT_setting1.pt
Pre-train model loaded! ../pretrained_models/PCT_setting1.pt


In [3]:
class usage():
    def __init__(self,K=3):
        self.K=K
        self.box_size_list=[]
        self.simulation_times, self.search_depth, self.rollout_length = (50, None, -1)
        self.mctree=None
        self.new=True
        self.action=None
        
        args = get_args()
        self.env = gym.make(args.id,
                           setting = args.setting,
                           item_set = args.item_size_set,
                           container_size=args.container_size,
                           data_name=args.dataset_path,
                           load_test_data=args.load_dataset,
                           internal_node_holder = args.internal_node_holder,
                           leaf_node_holder = args.leaf_node_holder,
                           LNES = args.lnes,
                           shuffle=args.shuffle,
                           sample_from_distribution=args.sample_from_distribution,
                           sample_left_bound=args.sample_left_bound,
                           sample_right_bound=args.sample_right_bound
                           )
        if args.no_cuda:
            self.device = torch.device('cpu')
        else:
            self.device = torch.device('cuda', args.device)
            torch.cuda.set_device(self.device)
        self.reset()
    def reset(self):
        self.box_size_list=[]
        self.mctree=None
        self.new=True
        self.action=None
        self.obs=self.env.reset()
        self.env.box_creator.preview(1000)
        
    def go(self,box):
        if len(self.box_size_list)<self.K-1: #1~K-1
            self.env.box_creator.box_list[len(self.box_size_list)]=box
            self.box_size_list.append(box)
            return False,None,None
        else: # K
            if self.new:
                self.env.box_creator.box_list[self.K-1]=box
                self.box_size_list.append(box)
                self.obs=self.env.cur_observation()
                self.mctree = MCTree(copy.deepcopy(self.env), copy.deepcopy(self.obs), self.box_size_list, search_depth=self.search_depth, rollout_length=self.rollout_length)
                self.new=False
            else:
                self.env.box_creator.box_list[self.K-1]=box
                self.box_size_list.pop(0)
                self.box_size_list.append(box)
                self.obs=self.env.cur_observation()
                self.mctree = MCTree(copy.deepcopy(self.env), copy.deepcopy(self.obs), self.box_size_list, search_depth=self.search_depth, rollout_length=self.rollout_length)
#                 self.env.box_creator.box_list[self.K-1]=box
#                 self.obs=self.env.cur_observation()
#                 self.mctree.succeed(self.action, box, self.obs)
            
            pl = self.mctree.get_policy(self.simulation_times, zeta=1e-5)
            act_node = self.mctree.sample_action(pl)
            
            # update env
            assert self.env.next_box == self.box_size_list[0]
            obs, r, done, dt = self.env.step(act_node, 'testMCTS')
            self.action = dt['box_action']
            print(self.action)

            if done:
                self.reset()
                return True,self.action,[dt['ratio'], dt['counter'], dt['reward']]
            else:
                return False,self.action,None

# Generate boxes sequence

In [10]:
import pickle
boxes = []
for _ in range(10000):
    box=(np.random.randint(1,6),np.random.randint(1,6),np.random.randint(1,6))
    boxes.append(box)
with open('box_seq.plk','wb') as f:
    pickle.dump(boxes,f)
    f.close()

In [4]:
import pickle
boxes = []
with open('box_seq.plk','rb') as f:
    boxes = pickle.load(f)
    f.close()

# experiment

In [5]:
import time
import csv
agent = usage(9)
results = []
actions = []
for no, box in enumerate(boxes):
    is_done,action,result=agent.go(box)
    if action != None:
        actions.append(action)
    if is_done:
        t = time.strftime('%Y.%m.%d-%H-%M-%S', time.localtime(time.time()))
        with open(f'actions/{t}.csv','w') as f:
            csv_writer = csv.writer(f)
            for act in actions:
                csv_writer.writerow(act)
            f.close()
        actions=[]
        
        print(result)
        results.append(result)
        if len(results)>=30:
            break

/root/anaconda3/envs/intelcup/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


tensor([[5]], device='cuda:0')
tensor([[5]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[26]], device='cuda:0')
tensor([[24]], device='cuda:0')
tensor([[42]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[7]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[35]], device='cuda:0')
tensor([[35]], device='cuda:0')
tensor([[8]], device='cuda:0')
tensor([[27]], device='cuda:0')
tensor([[17]], device='cuda:0')
tensor([[25]], device='cuda:0')
tensor([[20]], device='cuda:0')
tensor([[8]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[9]], device='cuda:0')
tensor([[9]], device='cuda:0')
tensor([[31]], device='cuda:0')
tensor([[24]], device='cuda:0')
tensor([[49]], device='cuda:0')
tensor([[17]], device='cuda:0')
tensor([[28]], device='cuda:0')
tensor([[46]], device='cuda:0')
tensor([[20]], device='cuda:0')
tensor([[20]], device='cuda:0')
tensor([[20]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[26]], devic

tensor([[36]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[48]], device='cuda:0')
tensor([[5]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[5]], device='cuda:0')
tensor([[33]], device='cuda:0')
tensor([[9]], device='cuda:0')
tensor([[9]], device='cuda:0')
tensor([[47]], device='cuda:0')
tensor([[28]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[17]], device='cuda:0')
tensor([[7]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[23]], device='cuda:0')
tensor([[23]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[44]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[15]], device='cuda:0')
tensor([[9]], device='cuda:0')
tensor([[17]], device='cuda:0')
tensor([[17]], device='cuda:0')
tensor([[26]], device='cuda:0')
tensor([[17]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[15]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[11]], devi

KeyboardInterrupt: 

In [ ]:
#3000 #90个容器的空间利用率

In [5]:
print(results)
mean = np.array(results).mean(axis=0)
print(mean)

[[0.824, 30, 8.24], [0.823, 34, 8.23], [0.802, 34, 8.02], [0.749, 36, 7.49], [0.827, 38, 8.27], [0.791, 30, 7.91], [0.767, 30, 7.67], [0.755, 29, 7.55], [0.767, 31, 7.67], [0.832, 36, 8.32], [0.826, 34, 8.26], [0.785, 26, 7.8500000000000005], [0.792, 28, 7.92], [0.867, 26, 8.67], [0.802, 30, 8.02], [0.805, 32, 8.05], [0.775, 35, 7.75], [0.826, 31, 8.26], [0.804, 28, 8.040000000000001], [0.813, 37, 8.129999999999999], [0.705, 23, 7.05], [0.822, 37, 8.219999999999999], [0.688, 32, 6.879999999999999], [0.838, 28, 8.379999999999999], [0.659, 25, 6.59], [0.914, 25, 9.14], [0.796, 28, 7.960000000000001], [0.902, 34, 9.02], [0.784, 31, 7.84], [0.783, 32, 7.83]]
[ 0.79743333 31.          7.97433333]


# Usage K=1

In [ ]:

class usage():
    def __init__(self,K=3):
        self.K=K
        self.box_size_list=[]
        self.simulation_times, self.search_depth, self.rollout_length = (50, None, -1)
        self.mctree=None
        self.new=True
        self.action=None
        
        args = get_args()
        self.env = gym.make(args.id,
                           setting = args.setting,
                           item_set = args.item_size_set,
                           container_size=args.container_size,
                           data_name=args.dataset_path,
                           load_test_data=args.load_dataset,
                           internal_node_holder = args.internal_node_holder,
                           leaf_node_holder = args.leaf_node_holder,
                           LNES = args.lnes,
                           shuffle=args.shuffle,
                           sample_from_distribution=args.sample_from_distribution,
                           sample_left_bound=args.sample_left_bound,
                           sample_right_bound=args.sample_right_bound
                           )
        if args.no_cuda:
            self.device = torch.device('cpu')
        else:
            self.device = torch.device('cuda', args.device)
            torch.cuda.set_device(self.device)
        self.reset()
    def reset(self):
        self.box_size_list=[]
        self.mctree=None
        self.new=True
        self.action=None
        self.obs=self.env.reset()
        self.env.box_creator.preview(1000)
        
    def go(self,box):
        if len(self.box_size_list)<self.K-1: #1~K-1
            self.env.box_creator.box_list[len(self.box_size_list)]=box
            self.box_size_list.append(box)
            return False,None,None
        else: # K
            if self.new:
                self.env.box_creator.box_list[self.K-1]=box
                self.box_size_list.append(box)
                self.obs=self.env.cur_observation()
                self.mctree = MCTree(copy.deepcopy(self.env), copy.deepcopy(self.obs), self.box_size_list, search_depth=self.search_depth, rollout_length=self.rollout_length)
                self.new=False
            else:
                self.env.box_creator.box_list[self.K-1]=box
                self.box_size_list.pop(0)
                self.box_size_list.append(box)
                self.obs=self.env.cur_observation()
                self.mctree = MCTree(copy.deepcopy(self.env), copy.deepcopy(self.obs), self.box_size_list, search_depth=self.search_depth, rollout_length=self.rollout_length)
#                 self.env.box_creator.box_list[self.K-1]=box
#                 self.obs=self.env.cur_observation()
#                 self.mctree.succeed(self.action, box, self.obs)
            
            pl = self.mctree.get_policy(self.simulation_times, zeta=1e-5)
            act_node = self.mctree.sample_action(pl)
            
            # update env
            assert self.env.next_box == self.box_size_list[0]
            obs, r, done, dt = self.env.step(act_node, 'testMCTS')
            self.action = dt['box_action']
            print(self.action)

            if done:
                self.reset()
                return True,self.action,[dt['ratio'], dt['counter'], dt['reward']]
            else:
                return False,self.action,None